In [54]:
class TreeTemplate:
    
    # the example below represents the borbón tree
    # not sure if I'll keep it all this way, but it's a good start
    
    
    def __init__(self, lsOfAges, sowDensityInt = 0): # use self to declare namespace
        self.ages = lsOfAges # ls of ages of the [insert tree name] trees
        self.trees = len(lsOfAges) # how many [insert tree name] there are
        self.sowDensity = sowDensityInt # sow density in trees/hectare
        
        self.firstHarvest = {'year': 4, 'proportion': 0.2} # year of first harvest and proportion of harvest until full
        self.fullHarvest = {'year': 5, 'proportion': 1.0}  # year of first harvest and proportion of harvest
        self.descentHarvest = {'year': 23, 'proportionDescent': 0.2} # year that production descends and annual proportion descent
        self.pruneHarvest = {'yearShift': -5, 'proportionAscent': 0.2} # these are placeholder values to remember to add member val
        self.death = {'year': 25} # year in which trees are expelled from dataset
        
        self.treeHarvestCap = 200 # units, in this case ?? lbs i think
        self.totalHarvest = 0 # units, in this case ???
        
        
    def addTreesAuto(self, number, ages = 0):
        """
        
        Parameters
        ----------
        self : class
            required to change-by-reference members of the class
            
        number: int
            the number of trees the user will add to the set
            
        ages : int
            the age of the trees (which translates to the element in the list)
            the user will add to the set. default value is 0 because most
            trees are planted/added as seeds/saplings, however the param
            is adjustable because it might be useful if adding new land with
            existing trees.
            
        """
        for i in range(len(number)):
            self.ages.append(ages) # all trees planted begin at age 0
            
    def addTreeSet(self, ls):
        """
        
        Parameters
        ----------
        self : class
            required to change-by-reference members of the class
            
        ls : list
            a list (of int) where each element represents a 'tree,' with
            the element's value representing that specific tree's age
            
        """
        for i,e in enumerate(ls):
            self.ages.append(e)
            
    # this may be a subfunciton in the 'addYears' function
    def oneYear(self):
        """
        
        This function takes this entire set of trees and adjusts the member values in the class to grow/change/produce accordingly.
        The function uses preset parameters for the specific tree type to guide the flow-control.
        
        """
        for treeIndex, treeAge in enumerate(self.ages):
            if (treeAge < self.firstHarvest['year']):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                self.totalHarvest += self.treeHarvestCap * self.firstHarvest['proportion']
                self.ages[treeIndex] += 1 # assure to reference the list and not the copy
                
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
                
            elif ((treeAge >= self.fullHarvest['year']) and (treeAge < self.descentHarvest['year'])):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                self.totalHarvest += self.treeHarvestCap * self.fullHarvest['proportion']
                self.ages[treeIndex] += 1
                
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            elif ((treeAge >= self.descentHarvest['year']) and (treeAge) < self.death['year']):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                yearsIntoDescent = treeAge - self.descentHarvest['year']
                proportion = self.fullHarvest['proportion'] - (yearsIntoDescent * self.descentHarvest['proportionDescent'])
                self.totalHarvest += self.treeHarvestCap * proportion
                self.ages[treeIndex] += 1
                
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            elif (treeAge == self.death['year']):
                #print("Tree dies now")
                #print("Harvest before: ", self.totalHarvest)
                continue
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            else:
                print("""The number: %d, list index: %d is out of range:
                a tree can not be less than 0 years old, and a tree of this type can not be more than
                %d years of age"""%(treeAge, treeIndex, self.death['year']))
                break
                   
        self.ages[:] = [age for age in self.ages if (age >= 25)] # call-by-reference overwrite of ls removing dead trees. 
        # assure this ^ is outside of the loop & assure the list references the full index with '[:]'
                
    def setHarvestZero(self):
        self.totalHarvest = 0

In [51]:
def get_yn():
    loop = 1
    while loop:
        choice = input("Confirm ([Y]/N):")
        if choice in ['y', 'Y', '1', '']:
            return True
        elif choice in ['n', 'N', '0']:
            return False
        else:
            print("Invalid choice. Please try again!\n")


In [7]:
def findSowDensity():
    # do something
    sowDensity = 0

In [56]:
def promptUnits():
    print("Please select the units of measurement of the land in question:")
    print("\t(1) Cuerdas")
    print("\t(2) Hectares")
    print("\t(3) Square Kilometers")
    print("\t(4) Acres")
    print("\t(5) Square Miles")
    
    select = input("Enter a number between one and five: ")
    
    if (select == 1) or (select == '1') or (select == 'one'): # cuerdas to hectares
        cuerdas = input("Enter the # of cuerdas: ")
        hectares = unitsToHectares(1,cuerdas)
        
    elif (select == 2) or (select == '2') or (select == 'two'):
        hectares = input("Enter the # of hectares: ")
        
    elif (select == 3) or (select == '3') or (select == 'three'):
        kilos = input("Enter the # of square kilometers: ")
        hectares = unitsToHectares(3, kilos)
            
    elif (select == 4) or (select == '4') or (select == 'four'):
        acres = input("Enter the # of acres: ")
        hectares = unitsToHectares(4, acres)
                
    elif (select == 5) or (select == '5') or (select == 'five'):
        miles = input("enter the # of miles: ")
        hectares = unitsToHectares(5, miles)
        
    else:
        print("Invalid input. Try again.")
    
    return(hectares)

In [ ]:
def unitsToHectares(tipo, units):
    
    """
    This function is utilized in user input function, but is also function-al within code.
    
    """
    if (tipo == 1) or (tipo == '1'): # cuerdas to hectares
        hectares = units * 3
        #hectares * 0.393???
        
    elif (tipo == 2) or (tipo == '2'):
        hectares = units
        
    elif (tipo == 3) or (tipo == '3'):
        hectares = 100 * units
            
    elif (tipo == 4) or (tipo == '4'):
        hectares = units / 2.471
                
    elif (tipo == 5) or (tipo == '5'):
        hectares = units * 258.999
        
    return(hectares)